In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn as nn

# Define the Generator (GAN) model for denoising
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, 3, kernel_size=3, stride=1, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

# Load pre-trained GAN model
model = Generator()
model.load_state_dict(torch.load('path_to_pretrained_gan.pth', map_location=torch.device('cpu')))
model.eval()

# Define transform to convert image to tensor
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

# Load fashion photograph
fashion_image_path = 'path_to_fashion_photograph.jpg'
fashion_image = Image.open(fashion_image_path).convert('RGB')

# Apply transformation
image_tensor = transform(fashion_image).unsqueeze(0)

# Denoise the image using the GAN
with torch.no_grad():
    denoised_image = model(image_tensor)

# Convert the denoised tensor back to an image
denoised_image = denoised_image.squeeze(0).permute(1, 2, 0).clamp(0, 1).numpy()

# Display the original and denoised images
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.title('Original Fashion Photograph')
plt.imshow(fashion_image)
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title('Denoised Fashion Photograph (Using GAN)')
plt.imshow(denoised_image)
plt.axis('off')

plt.show()
